In [3]:
source="java"
target="cs"
lr=1e-4
batch_size=8
beam_size=10
source_length=320
target_length=256
output_dir=f"saved_models/{source}-{target}/"
train_file=f"data/train.java-cs.txt.{source},data/train.java-cs.txt.{target}"
dev_file=f"data/valid.java-cs.txt.{source},data/valid.java-cs.txt.{target}"
epochs=100
pretrained_model="microsoft/graphcodebert-base"

In [2]:
# !git clone https://github.com/microsoft/CodeBERT.git

In [3]:
# !pip install torch
# !pip install transformers
# !pip install tree_sitter

In [4]:
# !cd GraphCodeBERT/translation
# !cd parser
# !bash build.sh
# !cd ..
# !unzip data.zip

In [4]:
from __future__ import absolute_import
import os
import sys
import pickle
import torch
import json
import random
import logging
import argparse
import numpy as np
from io import open
from itertools import cycle
import torch.nn as nn
from model import Seq2Seq
from tqdm import tqdm, trange
from bleu import _bleu
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from torch.utils.data.distributed import DistributedSampler
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaModel, RobertaTokenizer)
MODEL_CLASSES = {'roberta': (RobertaConfig, RobertaModel, RobertaTokenizer)}

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
from parser import DFG_python,DFG_java,DFG_ruby,DFG_go,DFG_php,DFG_javascript,DFG_csharp
from parser import (remove_comments_and_docstrings,
                   tree_to_token_index,
                   index_to_code_token,
                   tree_to_variable_index)
from tree_sitter import Language, Parser

In [5]:
dfg_function={
    'python':DFG_python,
    'java':DFG_java,
    'ruby':DFG_ruby,
    'go':DFG_go,
    'php':DFG_php,
    'javascript':DFG_javascript,
    'c_sharp':DFG_csharp,
}

logger = logging.getLogger(__name__)
#load parsers
parsers={}        
for lang in dfg_function:
    # print(Language)
    LANGUAGE = Language('parser/my-languages.so', lang)
    parser = Parser()
    parser.set_language(LANGUAGE) 
    parser = [parser,dfg_function[lang]]    
    parsers[lang]= parser

/home/judy/miniconda3/envs/fisc/lib/python3.12/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


### Helper functions

In [6]:
class Example(object):
    """A single training/test example."""
    def __init__(self,
                 source,
                 target,
                 lang
                 ):
        self.source = source
        self.target = target
        self.lang=lang

In [7]:
def extract_dataflow(code, parser,lang):
    #remove comments
    try:
        code=remove_comments_and_docstrings(code,lang)
    except:
        pass    
    #obtain dataflow
    if lang=="php":
        code="<?php"+code+"?>"    
    try:
        tree = parser[0].parse(bytes(code,'utf8'))    
        root_node = tree.root_node  
        tokens_index=tree_to_token_index(root_node)     
        code=code.split('\n')
        code_tokens=[index_to_code_token(x,code) for x in tokens_index]  
        index_to_code={}
        for idx,(index,code) in enumerate(zip(tokens_index,code_tokens)):
            index_to_code[index]=(idx,code)  
        try:
            DFG,_=parser[1](root_node,index_to_code,{}) 
        except:
            DFG=[]
        DFG=sorted(DFG,key=lambda x:x[1])
        indexs=set()
        for d in DFG:
            if len(d[-1])!=0:
                indexs.add(d[1])
            for x in d[-1]:
                indexs.add(x)
        new_DFG=[]
        for d in DFG:
            if d[1] in indexs:
                new_DFG.append(d)
        dfg=new_DFG
    except:
        dfg=[]
    return code_tokens,dfg

In [8]:
def read_examples(filename):
    """Read examples from filename."""
    examples=[]
    source,target=filename.split(',')
    lang='java'
    if source[-1]=='s':
        lang='c_sharp'
        
    with open(source,encoding="utf-8") as f1,open(target,encoding="utf-8") as f2:
        for line1,line2 in zip(f1,f2):
            line1=line1.strip()
            line2=line2.strip()
            examples.append(
                Example(
                    source=line1,
                    target=line2,
                    lang=lang
                        ) 
            )

    return examples

**Define the input for BERT model**

In [9]:
class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 example_id,
                 source_ids,
                 position_idx,
                 dfg_to_code,
                 dfg_to_dfg,                 
                 target_ids,
                 source_mask,
                 target_mask,

    ):
        self.example_id = example_id
        self.source_ids = source_ids
        self.position_idx = position_idx
        self.dfg_to_code = dfg_to_code
        self.dfg_to_dfg = dfg_to_dfg
        self.target_ids = target_ids
        self.source_mask = source_mask
        self.target_mask = target_mask       
        

In [10]:
def convert_examples_to_features(examples, tokenizer, args,stage=None):
    features = []
    for example_index, example in enumerate(tqdm(examples,total=len(examples))):
        ##extract data flow
        code_tokens,dfg=extract_dataflow(example.source,
                                         parsers["c_sharp" if args.source_lang == "cs" else "java"],
                                         "c_sharp" if args.source_lang == "cs" else "java")
        code_tokens=[tokenizer.tokenize('@ '+x)[1:] if idx!=0 else tokenizer.tokenize(x) for idx,x in enumerate(code_tokens)]
        ori2cur_pos={}
        ori2cur_pos[-1]=(0,0)
        for i in range(len(code_tokens)):
            ori2cur_pos[i]=(ori2cur_pos[i-1][1],ori2cur_pos[i-1][1]+len(code_tokens[i]))    
        code_tokens=[y for x in code_tokens for y in x]  
        
        #truncating
        code_tokens=code_tokens[:args.max_source_length-3][:512-3]
        source_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
        source_ids =  tokenizer.convert_tokens_to_ids(source_tokens)
        position_idx = [i+tokenizer.pad_token_id + 1 for i in range(len(source_tokens))]
        dfg=dfg[:args.max_source_length-len(source_tokens)]
        source_tokens+=[x[0] for x in dfg]
        position_idx+=[0 for x in dfg]
        source_ids+=[tokenizer.unk_token_id for x in dfg]
        padding_length=args.max_source_length-len(source_ids)
        position_idx+=[tokenizer.pad_token_id]*padding_length
        source_ids+=[tokenizer.pad_token_id]*padding_length      
        source_mask = [1] * (len(source_tokens))
        source_mask+=[0]*padding_length        
        
        #reindex
        reverse_index={}
        for idx,x in enumerate(dfg):
            reverse_index[x[1]]=idx
        for idx,x in enumerate(dfg):
            dfg[idx]=x[:-1]+([reverse_index[i] for i in x[-1] if i in reverse_index],)    
        dfg_to_dfg=[x[-1] for x in dfg]
        dfg_to_code=[ori2cur_pos[x[1]] for x in dfg]
        length=len([tokenizer.cls_token])
        dfg_to_code=[(x[0]+length,x[1]+length) for x in dfg_to_code]        
      

        #target
        if stage=="test":
            target_tokens = tokenizer.tokenize("None")
        else:
            target_tokens = tokenizer.tokenize(example.target)[:args.max_target_length-2]
        target_tokens = [tokenizer.cls_token]+target_tokens+[tokenizer.sep_token]            
        target_ids = tokenizer.convert_tokens_to_ids(target_tokens)
        target_mask = [1] *len(target_ids)
        padding_length = args.max_target_length - len(target_ids)
        target_ids+=[tokenizer.pad_token_id]*padding_length
        target_mask+=[0]*padding_length   
   
        if example_index < 5:
            if stage=='train':
                logger.info("*** Example ***")
                logger.info("source_tokens: {}".format([x.replace('\u0120','_') for x in source_tokens]))
                logger.info("source_ids: {}".format(' '.join(map(str, source_ids))))
                logger.info("source_mask: {}".format(' '.join(map(str, source_mask))))
                logger.info("position_idx: {}".format(position_idx))
                logger.info("dfg_to_code: {}".format(' '.join(map(str, dfg_to_code))))
                logger.info("dfg_to_dfg: {}".format(' '.join(map(str, dfg_to_dfg))))
                
                logger.info("target_tokens: {}".format([x.replace('\u0120','_') for x in target_tokens]))
                logger.info("target_ids: {}".format(' '.join(map(str, target_ids))))
                logger.info("target_mask: {}".format(' '.join(map(str, target_mask))))
       
        features.append(
            InputFeatures(
                 example_index,
                 source_ids,
                 position_idx,
                 dfg_to_code,
                 dfg_to_dfg,
                 target_ids,
                 source_mask,
                 target_mask,
            )
        )
    return features

## Starting fine-tuning

In [23]:
import argparse

# Simulate argparse for Jupyter
class Args:
    model_type = "roberta"
    model_name_or_path = "roberta-base"
    output_dir = "./output"
    load_model_path = None
    train_filename = None
    dev_filename = None
    test_filename = None
    source_lang = "en"
    config_name = ""
    tokenizer_name = ""
    max_source_length = 64
    max_target_length = 32
    do_train = True
    do_eval = True
    do_test = False
    do_lower_case = False
    no_cuda = False
    train_batch_size = 8
    eval_batch_size = 256
    gradient_accumulation_steps = 1
    learning_rate = 5e-5
    beam_size = 10
    weight_decay = 0.0
    adam_epsilon = 1e-8
    max_grad_norm = 1.0
    num_train_epochs = 3
    max_steps = -1
    eval_steps = -1
    train_steps = -1
    warmup_steps = 0
    local_rank = -1
    seed = 42

# Create an instance of Args
args = Args()

# Print the arguments for verification
print(f"Arguments: {args}")
args.model_type = "roberta"
args.source_lang = source
args.target_lang = target
args.model_name_or_path = pretrained_model
args.tokenizer_name = "microsoft/graphcodebert-base"
args.config_name = "microsoft/graphcodebert-base"
args.train_filename = train_file
args.dev_filename = dev_file
args.output_dir = output_dir
args.learning_rate = lr
args.num_train_epochs = epochs
args.train_batch_size = batch_size
args.eval_batch_size = batch_size
args.max_source_length = source_length
args.max_target_length = target_length

Arguments: <__main__.Args object at 0x7d6424a3ea20>


In [12]:
# prepare parsers for each language
parsers={}        
for lang in dfg_function:
    LANGUAGE = Language('parser/my-languages.so', lang)
    parser = Parser()
    parser.set_language(LANGUAGE) 
    parser = [parser,dfg_function[lang]]    
    parsers[lang]= parser
    

In [13]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYHTONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
set_seed(args.seed)

Define dataset

In [14]:
class TextDataset(Dataset):
    def __init__(self, examples, args):
        self.examples = examples
        self.args=args  
        
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, item):
        #calculate graph-guided masked function
        attn_mask=np.zeros((self.args.max_source_length,self.args.max_source_length),dtype=np.bool)
        #calculate begin index of node and max length of input
        node_index=sum([i>1 for i in self.examples[item].position_idx])
        max_length=sum([i!=1 for i in self.examples[item].position_idx])
        #sequence can attend to sequence
        attn_mask[:node_index,:node_index]=True
        #special tokens attend to all tokens
        for idx,i in enumerate(self.examples[item].source_ids):
            if i in [0,2]:
                attn_mask[idx,:max_length]=True
        #nodes attend to code tokens that are identified from
        for idx,(a,b) in enumerate(self.examples[item].dfg_to_code):
            if a<node_index and b<node_index:
                attn_mask[idx+node_index,a:b]=True
                attn_mask[a:b,idx+node_index]=True
        #nodes attend to adjacent nodes         
        for idx,nodes in enumerate(self.examples[item].dfg_to_dfg):
            for a in nodes:
                if a+node_index<len(self.examples[item].position_idx):
                    attn_mask[idx+node_index,a+node_index]=True  
                    
        return (torch.tensor(self.examples[item].source_ids),
                torch.tensor(self.examples[item].source_mask),
                torch.tensor(self.examples[item].position_idx),
                torch.tensor(attn_mask), 
                torch.tensor(self.examples[item].target_ids),
                torch.tensor(self.examples[item].target_mask),)
    


In [15]:
# Setup CUDA, GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()
args.device = device

In [16]:
# make dir if output_dir not exist
if os.path.exists(args.output_dir) is False:
    os.makedirs(args.output_dir)

In [17]:
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
config = config_class.from_pretrained(args.config_name)
tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name )
print(f"tokenizer: {tokenizer}")

tokenizer: RobertaTokenizer(name_or_path='microsoft/graphcodebert-base', vocab_size=50265, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}


In [19]:
# build model
encoder = model_class.from_pretrained(args.model_name_or_path,config=config)    
decoder_layer = nn.TransformerDecoderLayer(d_model=config.hidden_size, nhead=config.num_attention_heads)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
model=Seq2Seq(encoder=encoder,decoder=decoder,config=config,
                beam_size=args.beam_size,max_length=args.max_target_length,
                sos_id=tokenizer.cls_token_id,eos_id=tokenizer.sep_token_id)

if args.load_model_path is not None:
    logger.info("reload model from {}".format(args.load_model_path))
    model.load_state_dict(torch.load(args.load_model_path))
    
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Seq2Seq(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [1]:
# wrap this code to pytorch lightning
import pytorch_lightning as pl
import torch
from torch import nn
from transformers import AutoModel, AutoTokenizer

class Seq2SeqLightning(pl.LightningModule):
    def __init__(self, model_class, model_name_or_path, config, tokenizer, args):
        super().__init__()
        self.save_hyperparameters()
        
        # Encoder
        self.encoder = model_class.from_pretrained(model_name_or_path, config=config)
        # Decoder
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=config.hidden_size, 
            nhead=config.num_attention_heads
        )
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
        
        # Seq2Seq Model
        self.model = Seq2Seq(
            encoder=self.encoder,
            decoder=self.decoder,
            config=config,
            beam_size=args.beam_size,
            max_length=args.max_target_length,
            sos_id=tokenizer.cls_token_id,
            eos_id=tokenizer.sep_token_id
        )
        
        # Load pre-trained weights if specified
        if args.load_model_path is not None:
            self.log(f"Reloading model from {args.load_model_path}")
            state_dict = torch.load(args.load_model_path, map_location=self.device)
            self.model.load_state_dict(state_dict)
        
        # self.args = args                                                                                                  
        
    def forward(self, *args, **kwargs):
        return self.model(*args, **kwargs)

    def training_step(self, batch, batch_idx):
        # loss = self.model(**batch)  # Assuming the model returns the loss
        batch = tuple(t.to(device) for t in batch)
        source_ids,source_mask,position_idx,att_mask,target_ids,target_mask = batch
        loss, _, _ = model(source_ids,source_mask,position_idx,att_mask,target_ids,target_mask)
        self.log("train_loss", loss)
        return loss

    # def validation_step(self, batch, batch_idx):
    #     loss = self.model(**batch)  # Assuming the model returns the loss
    #     self.log("val_loss", loss)
    #     return loss
    
    def validation_step(self, batch, batch_idx):
        # Initialize evaluation results and flags
        eval_loss, tokens_num = 0, 0
        eval_ppl, dev_bleu, xmatch = None, None, None

        # Unpack the batch
        source_ids, source_mask, position_idx, att_mask, target_ids, target_mask = batch

        # Forward pass for loss calculation
        with torch.no_grad():
            _, loss, num = self.model(
                source_ids=source_ids,
                source_mask=source_mask,
                position_idx=position_idx,
                att_mask=att_mask,
                target_ids=target_ids,
                target_mask=target_mask,
            )

        # Accumulate evaluation metrics
        eval_loss += loss.sum().item()
        tokens_num += num.sum().item()

        # Compute perplexity
        eval_loss = eval_loss / tokens_num
        eval_ppl = round(np.exp(eval_loss), 5)

        # Log validation loss and perplexity
        self.log("val_loss", eval_loss, prog_bar=True, logger=True)
        self.log("val_ppl", eval_ppl, prog_bar=True, logger=True)

        # BLEU and xMatch evaluation
        predictions = []
        accs = []
        with torch.no_grad():
            preds = self.model(source_ids, source_mask, position_idx, att_mask)
            for pred in preds:
                t = pred[0].cpu().numpy().tolist()
                if 0 in t:
                    t = t[:t.index(0)]
                text = self.tokenizer.decode(t, clean_up_tokenization_spaces=False)
                predictions.append(text)

        # Compare predictions with ground truth
        for pred, gold in zip(predictions, batch["target_text"]):  # Assuming ground truth is in batch["target_text"]
            accs.append(pred == gold)

        # Save predictions and ground truth for BLEU calculation
        dev_bleu = round(
            _bleu(predictions, batch["target_text"]), 2
        )  # Replace `_bleu` with your BLEU scoring implementation
        xmatch = round(np.mean(accs) * 100, 4)

        # Log BLEU and xMatch scores
        self.log("val_bleu", dev_bleu, prog_bar=True, logger=True)
        self.log("val_xmatch", xmatch, prog_bar=True, logger=True)

        # Save best model checkpoint based on BLEU+xMatch
        metric = dev_bleu + xmatch
        if metric > self.best_metric:
            self.best_metric = metric
            self.save_checkpoint("best_bleu", self.model)

        return {
            "val_loss": eval_loss,
            "val_ppl": eval_ppl,
            "val_bleu": dev_bleu,
            "val_xmatch": xmatch,
        }


    # def test_step(self, batch, batch_idx):
    #     loss = self.model(**batch)  # Assuming the model returns the loss
    #     self.log("test_loss", loss)
    #     return loss
    def test_step(self, batch, batch_idx):
        p = []  # Predictions list
        source_ids, source_mask, position_idx, att_mask, target_ids, target_mask = batch

        # Generate predictions
        with torch.no_grad():
            preds = self.model(source_ids, source_mask, position_idx, att_mask)
            for pred in preds:
                t = pred[0].cpu().numpy().tolist()
                if 0 in t:
                    t = t[:t.index(0)]
                text = self.tokenizer.decode(t, clean_up_tokenization_spaces=False)
                p.append(text)

        # Compute BLEU and Exact Match (xMatch)
        predictions, accs = [], []
        for ref, gold in zip(p, batch["target_text"]):  # Assuming ground truth is in batch["target_text"]
            predictions.append(ref)
            accs.append(ref == gold)

        # Save predictions and ground truth
        output_file = os.path.join(self.hparams.args.output_dir, f"test_{batch_idx}.output")
        gold_file = os.path.join(self.hparams.args.output_dir, f"test_{batch_idx}.gold")

        with open(output_file, "w") as f_pred, open(gold_file, "w") as f_gold:
            for pred, gold in zip(predictions, batch["target_text"]):
                f_pred.write(pred + "\n")
                f_gold.write(gold + "\n")

        # Calculate BLEU score
        dev_bleu = round(
            _bleu(gold_file, output_file), 2
        )  # Replace `_bleu` with your BLEU calculation logic

        # Log metrics
        xmatch = round(np.mean(accs) * 100, 4)
        self.log("test_bleu", dev_bleu, prog_bar=True, logger=True)
        self.log("test_xmatch", xmatch, prog_bar=True, logger=True)

        logger.info("  %s = %s " % ("bleu-4", str(dev_bleu)))
        logger.info("  %s = %s " % ("xMatch", str(xmatch)))
        logger.info("  " + "*" * 20)

        return {"test_bleu": dev_bleu, "test_xmatch": xmatch}


    def configure_optimizers(self, *args, **kwargs):
        # optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.args.learning_rate)
        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                'weight_decay': args.weight_decay},
            {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
        return optimizer
    
    def configure_scheduler(self):
        return get_linear_schedule_with_warmup(
            self.optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=self.total_steps
        )


In [20]:
if args.n_gpu > 1:
    # multi-gpu training
    model = torch.nn.DataParallel(model)

### ------- TRAINING -------

In [21]:
def eval(model, dev_dataset):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0                     
    eval_flag=False    
    if 'dev_loss' in dev_dataset:
        eval_examples,eval_data=dev_dataset['dev_loss']
    else:
        eval_examples = read_examples(args.dev_filename)
        eval_features = convert_examples_to_features(eval_examples, tokenizer, args, stage='dev')
        eval_data = TextDataset(eval_features,args)
        dev_dataset['dev_loss']=eval_examples,eval_data
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size,num_workers=4)

    logger.info("\n***** Running evaluation *****")
    logger.info("  Num examples = %d", len(eval_examples))
    logger.info("  Batch size = %d", args.eval_batch_size)

    #Start Evaling model
    model.eval()
    eval_loss,tokens_num = 0,0
    for batch in eval_dataloader:
        batch = tuple(t.to(device) for t in batch)               
        source_ids,source_mask,position_idx,att_mask,target_ids,target_mask = batch
        with torch.no_grad():
            _,loss,num = model(source_ids,source_mask,position_idx,att_mask,target_ids,target_mask)     
        eval_loss += loss.sum().item()
        tokens_num += num.sum().item()
    
    eval_loss = eval_loss / tokens_num
    model.train()
    return eval_loss

In [22]:
def save_model(model, output_dir, step):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model_to_save = model.module if hasattr(model, 'module') else model 
    output_model_file = os.path.join(output_dir, "model.{}.bin".format(step))
    torch.save(model_to_save.state_dict(), output_model_file)

In [23]:
# start training
model.train()
train_examples = read_examples(args.train_filename)
train_features = convert_examples_to_features(train_examples, tokenizer,args,stage='train')
train_data = TextDataset(train_features,args)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size//args.gradient_accumulation_steps,num_workers=54)
num_train_optimization_steps =  args.train_steps

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        'weight_decay': args.weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=len(train_dataloader)*args.num_train_epochs*0.1,num_training_steps=len(train_dataloader)*args.num_train_epochs)

#Start training
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", args.train_batch_size)
logger.info("  Num epoch = %d", args.num_train_epochs)

  0%|          | 0/10300 [00:00<?, ?it/s]11/17/2024 18:14:41 - INFO - __main__ -   *** Example ***
11/17/2024 18:14:41 - INFO - __main__ -   source_tokens: ['<s>', 'public', '_List', 'Spe', 'ech', 'Sy', 'nt', 'hesis', 'T', 'asks', 'Result', '_list', 'Spe', 'ech', 'Sy', 'nt', 'hesis', 'T', 'asks', '_', '_(', '_List', 'Spe', 'ech', 'Sy', 'nt', 'hesis', 'T', 'asks', 'Request', '_request', '_)', '_{', '_request', '_=', '_before', 'Client', 'Exec', 'ution', '_(', '_request', '_)', '_;', '_return', '_execute', 'List', 'Spe', 'ech', 'Sy', 'nt', 'hesis', 'T', 'asks', '_(', '_request', '_)', '_;', '_}', '</s>', 'request', 'request', 'request', 'beforeClientExecution', 'request', 'request']
11/17/2024 18:14:41 - INFO - __main__ -   source_ids: 0 15110 9527 29235 7529 35615 3999 35571 565 40981 48136 889 29235 7529 35615 3999 35571 565 40981 1437 36 9527 29235 7529 35615 3999 35571 565 40981 45589 2069 4839 25522 2069 5457 137 47952 46891 15175 36 2069 4839 25606 671 11189 36583 29235 7529 35615 

In [24]:
def calculate_bleu(model, dev_dataset):
    if 'dev_bleu' in dev_dataset:
        eval_examples,eval_data=dev_dataset['dev_bleu']
    else:
        eval_examples = read_examples(args.dev_filename)
        eval_examples = random.sample(eval_examples,min(1000,len(eval_examples)))
        eval_features = convert_examples_to_features(eval_examples, tokenizer, args,stage='test')
        eval_data = TextDataset(eval_features,args)
        dev_dataset['dev_bleu']=eval_examples,eval_data
        
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size,num_workers=4)
    model.eval() 
    p=[]
    for batch in eval_dataloader:
        batch = tuple(t.to(device) for t in batch)
        source_ids,source_mask,position_idx,att_mask,target_ids,target_mask = batch                 
        with torch.no_grad():
            preds = model(source_ids,source_mask,position_idx,att_mask)  
            for pred in preds:
                t=pred[0].cpu().numpy()
                t=list(t)
                if 0 in t:
                    t=t[:t.index(0)]
                text = tokenizer.decode(t,clean_up_tokenization_spaces=False)
                p.append(text)
    model.train()
    predictions=[]
    accs = []
    with open(os.path.join(args.output_dir,"dev.output"),'w') as f, open(os.path.join(args.output_dir,"dev.gold"),'w') as f1:
        for ref,gold in zip(p,eval_examples):
            predictions.append(ref)
            f.write(ref+'\n')
            f1.write(gold.target+'\n')     
            accs.append(ref==gold.target)

    dev_bleu=round(_bleu(os.path.join(args.output_dir, "dev.gold"), os.path.join(args.output_dir, "dev.output")),2)
    xmatch=round(np.mean(accs)*100,4)
    return dev_bleu,xmatch
    

In [30]:
def test(model):
    files=[]
    if args.dev_filename is not None:
        files.append(args.dev_filename)
    # if args.test_filename is not None:
    #     files.append(args.test_filename)
    for idx,file in enumerate(files):   
        logger.info("Test file: {}".format(file))
        eval_examples = read_examples(file)
        eval_features = convert_examples_to_features(eval_examples, tokenizer, args,stage='test')
        eval_data = TextDataset(eval_features,args) 

        # Calculate bleu
        eval_sampler = SequentialSampler(eval_data)
        eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size,num_workers=4)

        model.eval() 
        p=[]
        for batch in tqdm(eval_dataloader,total=len(eval_dataloader)):
            batch = tuple(t.to(device) for t in batch)
            source_ids,source_mask,position_idx,att_mask,target_ids,target_mask = batch                    
            with torch.no_grad():
                preds = model(source_ids,source_mask,position_idx,att_mask)  
                for pred in preds:
                    t=pred[0].cpu().numpy()
                    t=list(t)
                    if 0 in t:
                        t=t[:t.index(0)]
                    text = tokenizer.decode(t,clean_up_tokenization_spaces=False)
                    p.append(text)
        model.train()
        predictions=[]
        accs=[]
        with open(os.path.join(args.output_dir,"test_{}.output".format(str(idx))),'w') as f, open(os.path.join(args.output_dir,"test_{}.gold".format(str(idx))),'w') as f1:
            for ref,gold in zip(p,eval_examples):
                predictions.append(ref)
                f.write(ref+'\n')
                f1.write(gold.target+'\n')    
                accs.append(ref==gold.target)
        dev_bleu=round(_bleu(os.path.join(args.output_dir, "test_{}.gold".format(str(idx))).format(file), 
                                os.path.join(args.output_dir, "test_{}.output".format(str(idx))).format(file)),2)
        logger.info("  %s = %s "%("bleu-4",str(dev_bleu)))
        logger.info("  %s = %s "%("xMatch",str(round(np.mean(accs)*100,4))))
        logger.info("  "+"*"*20)   

In [26]:
# model.train()
# dev_dataset={}
# nb_tr_examples, nb_tr_steps,tr_loss,global_step,best_bleu,best_loss = 0, 0,0,0,0,1e6 

# for epoch in range(args.num_train_epochs):
#     bar = tqdm(train_dataloader,total=len(train_dataloader))
#     for batch in bar:
#         batch = tuple(t.to(device) for t in batch)
#         source_ids,source_mask,position_idx,att_mask,target_ids,target_mask = batch
#         loss,_,_ = model(source_ids,source_mask,position_idx,att_mask,target_ids,target_mask)

#         if args.n_gpu > 1:
#             loss = loss.mean() # mean() to average on multi-gpu.
#         if args.gradient_accumulation_steps > 1:
#             loss = loss / args.gradient_accumulation_steps
            
#         tr_loss += loss.item()
#         train_loss=round(tr_loss*args.gradient_accumulation_steps/(nb_tr_steps+1),4)
#         bar.set_description("epoch {} loss {}".format(epoch,train_loss))
#         nb_tr_examples += source_ids.size(0)
#         nb_tr_steps += 1
#         loss.backward()

#         if (nb_tr_steps + 1) % args.gradient_accumulation_steps == 0:
#             #Update parameters
#             optimizer.step()
#             optimizer.zero_grad()
#             scheduler.step()
#             global_step += 1
    
#     # do eval 
#     if epoch in [ int(args.num_train_epochs*(i+1)//20) for i in range(20)]:
#         eval_loss = eval(model, dev_dataset)
#         result = {'eval_ppl': round(np.exp(eval_loss),5),
#                     'global_step': global_step+1,
#                     'train_loss': round(train_loss,5)}
#         for key in sorted(result.keys()):
#             logger.info("  %s = %s", key, str(result[key]))
#         logger.info("  "+"*"*20)   
        
#         # save last checkpoint
#         last_output_dir = os.path.join(args.output_dir, 'checkpoint-last')
#         if not os.path.exists(last_output_dir):
#             os.makedirs(last_output_dir)
#         model_to_save = model.module if hasattr(model, 'module') else model
#         output_model_file = os.path.join(last_output_dir, "model.{}.bin".format(global_step))
#         torch.save(model_to_save.state_dict(), output_model_file)
#         if eval_loss<best_loss:
#             best_loss=eval_loss
#             best_output_dir = os.path.join(args.output_dir, 'checkpoint-best-ppl')
#             save_model(model, best_output_dir, global_step)
#             logger.info("  Best checkpoint is saved in %s",best_output_dir)
#             logger.info("  "+"*"*20)
        
#         # calculate bleu
#         dev_bleu, xmatch = calculate_bleu(model, dev_dataset)
#         logger.info("  %s = %s "%("bleu-4",str(dev_bleu)))
#         logger.info("  %s = %s "%("xMatch",str(xmatch)))
#         logger.info("  "+"*"*20)    
#         if dev_bleu+xmatch>best_bleu:
#             logger.info("  Best BLEU+xMatch:%s",dev_bleu+xmatch)
#             logger.info("  "+"*"*20)
#             best_bleu=dev_bleu+xmatch
#             # Save best checkpoint for best bleu
#             output_dir = os.path.join(args.output_dir, 'checkpoint-best-bleu')
#             if not os.path.exists(output_dir):
#                 os.makedirs(output_dir)
#             model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
#             output_model_file = os.path.join(output_dir, "pytorch_model.bin")
#             torch.save(model_to_save.state_dict(), output_model_file)
# test(model)
        

In [31]:
# load model and push to huggingface
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer
import torch
import os
import logging
import argparse
import numpy as np
from io import open
from itertools import cycle
import torch.nn as nn
from model import Seq2Seq
from tqdm import tqdm, trange
from bleu import _bleu
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from torch.utils.data.distributed import DistributedSampler

path = "/home/judy/code/code-transformer/CodeBERT/GraphCodeBERT/translation/saved_models/java-cs/checkpoint-best-ppl/pytorch_model.bin"
config = RobertaConfig.from_pretrained("microsoft/graphcodebert-base")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/graphcodebert-base")
model = Seq2Seq(encoder=RobertaModel.from_pretrained("microsoft/graphcodebert-base",config=config),
                decoder=nn.TransformerDecoder(nn.TransformerDecoderLayer(d_model=config.hidden_size, nhead=config.num_attention_heads), num_layers=6),
                config=config,beam_size=10,max_length=256,sos_id=tokenizer.cls_token_id,eos_id=tokenizer.sep_token_id)
model.load_state_dict(torch.load(path))



Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_53684/3119324797.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe

<All keys matched successfully>

In [ ]:
# test
model.to(device)
test(model)

11/17/2024 18:25:07 - INFO - __main__ -   Test file: data/valid.java-cs.txt.java,data/valid.java-cs.txt.cs
 21%|██        | 13/63 [01:35<05:13,  6.28s/it]

In [ ]:
# push to huggingface
!huggingface-cli login
repo_url = "https://huggingface.co/judynguyen16/graphcodebert--code-translation-java-cs"

## Training with Lightning Pytorch

In [18]:
# ---------*****---------
# Prepare training loader and fine-tuning
train_examples = read_examples(args.train_filename)
train_features = convert_examples_to_features(train_examples, tokenizer,args,stage='train')
train_data = TextDataset(train_features,args)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size//args.gradient_accumulation_steps,num_workers=54)
num_train_optimization_steps =  args.train_steps

#Start training
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", args.train_batch_size)
logger.info("  Num epoch = %d", args.num_train_epochs)

  0%|          | 0/10300 [00:00<?, ?it/s]11/20/2024 08:14:03 - INFO - __main__ -   *** Example ***
11/20/2024 08:14:03 - INFO - __main__ -   source_tokens: ['<s>', 'public', '_List', 'Spe', 'ech', 'Sy', 'nt', 'hesis', 'T', 'asks', 'Result', '_list', 'Spe', 'ech', 'Sy', 'nt', 'hesis', 'T', 'asks', '_', '_(', '_List', 'Spe', 'ech', 'Sy', 'nt', 'hesis', 'T', 'asks', 'Request', '_request', '_)', '_{', '_request', '_=', '_before', 'Client', 'Exec', 'ution', '_(', '_request', '_)', '_;', '_return', '_execute', 'List', 'Spe', 'ech', 'Sy', 'nt', 'hesis', 'T', 'asks', '_(', '_request', '_)', '_;', '_}', '</s>', 'request', 'request', 'request', 'beforeClientExecution', 'request', 'request']
11/20/2024 08:14:03 - INFO - __main__ -   source_ids: 0 15110 9527 29235 7529 35615 3999 35571 565 40981 48136 889 29235 7529 35615 3999 35571 565 40981 1437 36 9527 29235 7529 35615 3999 35571 565 40981 45589 2069 4839 25522 2069 5457 137 47952 46891 15175 36 2069 4839 25606 671 11189 36583 29235 7529 35615 

In [21]:
# validation loader
dev_dataset={}
if 'dev_loss' in dev_dataset:
    eval_examples,eval_data=dev_dataset['dev_loss']
else:
    eval_examples = read_examples(args.dev_filename)
    eval_features = convert_examples_to_features(eval_examples, tokenizer, args,stage='dev')
    eval_data = TextDataset(eval_features,args)
    dev_dataset['dev_loss']=eval_examples,eval_data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size,num_workers=54)

logger.info("\n***** Running evaluation *****")
logger.info("  Num examples = %d", len(eval_examples))
logger.info("  Batch size = %d", args.eval_batch_size)

100%|██████████| 500/500 [00:00<00:00, 642.84it/s]
11/20/2024 08:20:17 - INFO - __main__ -   
***** Running evaluation *****
11/20/2024 08:20:17 - INFO - __main__ -     Num examples = 500
11/20/2024 08:20:17 - INFO - __main__ -     Batch size = 8


In [24]:
# test loader
files=[]
if args.dev_filename is not None:
    files.append(args.dev_filename)
if args.test_filename is not None:
    files.append(args.test_filename)
for idx,file in enumerate(files):   
    logger.info("Test file: {}".format(file))
    eval_examples = read_examples(file)
    eval_features = convert_examples_to_features(eval_examples, tokenizer, args,stage='test')
    eval_data = TextDataset(eval_features,args) 

    # Calculate bleu
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size,num_workers=4)

11/20/2024 08:22:53 - INFO - __main__ -   Test file: data/valid.java-cs.txt.java,data/valid.java-cs.txt.cs
100%|██████████| 500/500 [00:00<00:00, 610.65it/s]
